## Correction of bollinger bands channel by RSX indicator

In [1]:
import pandas as pd
import numpy as np
from simple.chart import chartParallel, interactTable, chartProfit, chartTrades
from simple.funcs import vwap
from simple.jurik import JRSX
from simple.geneopt import GeneOpt
from simple.backtest import getProfit, npBacktestLimit
from multiprocessing import current_process

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
# declare non-default chart linestyles
line_styles = {
    'cA': dict(color='red', opacity=0.4),
    'cB': dict(color='green', opacity=0.4),
    'RSX': dict(color='orange', row=2)
}

In [4]:
def model(Period: int = (1000, 50000), StdDev: float = (1, 4, 0.1), 
          Threshold: int = (0, 20), Ratio: float = (0, 10, 0.1)):
    """Limit orders strategy by corrrected bollinger bands channel"""
    
    if Period == 0 or StdDev == 0: return {}
    Tick = T.Price
    Center = vwap(T, Period)
    _std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + _std * StdDev
    qB = Center - _std * StdDev
    RSX = JRSX(Tick, Period) - 50
    _ask_correction = Threshold - RSX
    _bid_correction = -Threshold - RSX
    cA = qA + _ask_correction * Ratio
    cB = qB + _bid_correction * Ratio

    _trades = npBacktestLimit(T, cA, cB)

    if current_process().daemon:
        P = getProfit(_trades)
        return {
            'Profit': P.Profit.sum(),
            'Count': len(P),
            'AvgProfit': P.Profit.mean() if len(P) > 0 else 0,
            'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
         }  
    else:
        return {**locals(), **chartProfit(_trades), **chartTrades(_trades)}

In [5]:
# Genetic optimizer
G = GeneOpt(model)
G.maximize(population_size=300, generations=5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'Period': 32830,
 'StdDev': 2.19062030509265,
 'Threshold': 7,
 'Ratio': 7.664531633187096}

In [6]:
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Profit', ascending=False).set_index('Period')
X

StdDev  Threshold     Ratio       Profit  Count   AvgProfit  \
Period                                                                  
32830   2.190620          7  7.664532  7244.391331   19.0  381.283754   
32371   2.236321          6  8.855149  7008.825200   18.0  389.379178   
33116   2.236158          5  8.293037  6983.388657   19.0  367.546771   
31692   2.190620          7  7.664532  6951.257856   19.0  365.855677   
30682   1.958148         10  6.601200  6942.759452   22.0  315.579975   
...          ...        ...       ...          ...    ...         ...   
0       1.958148         10  8.855149          NaN    NaN         NaN   
22936   0.000000         17  8.451753          NaN    NaN         NaN   
40445   0.000000          1  4.451096          NaN    NaN         NaN   
0       2.190620          9  6.771425          NaN    NaN         NaN   
0       2.008008          2  7.618445          NaN    NaN         NaN   

           Sharpe  
Period             
32830   12.518033  
32371   11.841704  
33116   11.996888  
31692   11.415450  
30682   15.097420  
...           ...  
0             NaN  
22936         NaN  
40445         NaN  
0             NaN  
0             NaN  

[928 rows x 7 columns]

In [9]:
# Optimization results browser
interactTable(model, X, height=600, rows=2, **line_styles)

In [8]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'Period',
                              'range': [0, 49887],
                              'values': array([32830, 32371, 33116, ..., 40445,     0,     0])},
                             {'label': 'StdDev',
                              'range': [0.0, 3.99818522140579],
                              'values': array([2.19062031, 2.23632131, 2.23615766, ..., 0.        , 2.19062031,
                                               2.00800789])},
                             {'label': 'Threshold',
                              'range': [0, 19],
                              'values': array([7, 6, 5, ..., 1, 9, 2])},
                             {'label': 'Ratio',
                              'range': [0.0, 9.98986350925396],
                              'values': array([7.66453163, 8.85514886, 8.29303722, ..., 4.4510958 , 6.77142454,
                                               7.61844526])},
                             {'label': 'Pr